In [ ]:
from api.resources.blender import Blender



In [1]:
import pandas as pd
import inspect
import bpy
# create a new geometry node tree
tree = bpy.data.node_groups.new(type='GeometryNodeTree', name='GeometryNodes')


# Create a new mesh data block
gridmesh = bpy.data.meshes.new('GridMesh')

if "Cube" in bpy.data.meshes:
    mesh = bpy.data.meshes["Cube"]
    print("removing mesh", mesh)
    bpy.data.meshes.remove(mesh)

# Create a new object that uses the mesh data block
obj = bpy.data.objects.new('Grid', gridmesh)
print(list(bpy.data.objects))

# Add the object to the scene collection
bpy.context.scene.collection.objects.link(obj)


print("\n--")
print("Estos son los objetos de tu escena")

    
# Leer el archivo CSV
print("\n--")
print("Este es tu Csv")
df = pd.read_csv('modelo.csv')
print(df)



separacion_muestras = 3
# Crear una visualización 3D de los datos
for i, row in df.iterrows():
    x, y, z, value = row['numeromuestra'], row['intensidad'], row['aceleracion'], row['altura']
    # Create a UV sphere

    mat = bpy.data.materials.new(name="SphereMaterial")
    mat.diffuse_color = (2**y, 0, 0, 0.5)  # Red color
    
    # Get the active object (which is the sphere we just created)
    bpy.ops.mesh.primitive_uv_sphere_add(location=(x*(separacion_muestras), y, 1), segments = z, ring_count = y, radius = value, calc_uvs = True) 
    sphere = bpy.context.active_object
    # Add a material to the sphere
    
    
    # Set the color of the material
    sphere.data.materials.append(mat)

# Exportar el modelo a SVG
output_file = "file.gltf"

# Set export options
export_settings = {
    "filepath": output_file,
    "check_existing": False,
    "export_colors": True,
    "export_animations": False,

}

# Export the active object
bpy.ops.export_scene.gltf(**export_settings)


removing mesh <bpy_struct, Mesh("Cube") at 0x7f80d2db8e08>

--
Estos son los objetos de tu escena

--
Este es tu Csv
   numeromuestra  intensidad  aceleracion  altura
0              1           4            1       1
1              2           3            6       1
2              3           2            1       1
3              4           1            2       1
4              1           4            2       1
5              5           1            2       1
6              5           2            4       1
7              7           1            2       1
8              8           1            2       1
9              4           1            2       1
19:51:30 | ERROR: Draco mesh compression is not available because library could not be found at /Users/macbookpro/Desktop/Elisa/apps/LIHO/liho/backend/Resources/3.4/python/lib/python3.10/site-packages/libextern_draco.dylib
19:51:30 | INFO: Starting glTF 2.0 export
19:51:30 | INFO: Extracting primitive: Sphere
19:51:30 | INFO: Primi

{'FINISHED'}

In [ ]:
import bpy
def create_material():
        

    # create a new material
    mat = bpy.data.materials.new(name="MySphereMaterial")

    # make the material red using nodes
    mat.use_nodes = True
    nodes = mat.node_tree.nodes
    for node in nodes:
        nodes.remove(node)

    diffuse_node = nodes.new(type='ShaderNodeBsdfDiffuse')
    diffuse_node.inputs[0].default_value = (1, 0, 0, 1)
    transparent_node = nodes.new(type='ShaderNodeBsdfTransparent')
    transparent_node.inputs[0].default_value = (1, 1, 1, 1)
    transparent_node.inputs[1].default_value = 0.5
    mix_node = nodes.new(type='ShaderNodeMixShader')
    mix_node.inputs[0].default_value = 0.5
    output_node = nodes.new(type='ShaderNodeOutputMaterial')
    links = mat.node_tree.links
    links.new(diffuse_node.outputs[0], mix_node.inputs[1])
    links.new(transparent_node.outputs[0], mix_node.inputs[2])
    links.new(mix_node.outputs[0], output_node.inputs[0])
    return mat



In [ ]:
row